# Model without `body` - AGAIN?!?
#### Making models based off the other features.

In [1]:
from ipynb.fs.full.functions import *

In [2]:
# Data to create our model
dfa = pd.read_csv('../data/ama_comments.csv')
dfb = pd.read_csv('../data/ar_comments.csv')
df = pd.concat([dfa, dfb], axis=0)
df = CleanUp(df).df.copy()

In [3]:
# Model X, and y
df = df.sample(n=df.shape[0], random_state=3)

X = df.drop(columns=['subreddit_binary'])#, 'body'])
# X = df['body']
y = df['subreddit_binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=3)

In [4]:
# TEST data (not part of train/test/split)
df1 = pd.read_csv('../data/2021-04-27_1812_AMA_comments.csv')
df2 = pd.read_csv('../data/2021-04-27_1812_AskReddit_comments.csv')
df_test_pred = pd.concat([df1, df2], axis=0)
df_test_pred = CleanUp(df_test_pred).df.copy()

In [5]:
df_test_pred = df_test_pred.sample(n=df_test_pred.shape[0], random_state=3)

X_new = df_test_pred.drop(columns=['subreddit_binary', 'body'])
# X_new = df_test_pred['body']
y_new = df_test_pred['subreddit_binary'] 

### MAKE MODEL FUNCTION(S)

In [6]:
print(f'{X_train.shape}, {X_test.shape}, {y_train.shape}, {y_test.shape}')

(7370, 8), (2457, 8), (7370,), (2457,)


In [7]:
print(f'{X.shape}, {X_new.shape}, {y.shape}, {y_new.shape}')

(9827, 8), (1962, 7), (9827,), (1962,)


# TRYING SOMETHING DIFFERENT:


In [10]:
tvect = TfidfVectorizer(max_features=5000)
X_tvect_train_body = tvect.fit_transform(X_train['body'])
X_tvect_test_body = tvect.transform(X_test['body'])

X_tvect_train = pd.concat([X_train, X_tvect_train_body]).reset_axis(drop=True)
X_tvect_test = pd.concat([X_test, X_tvect_test_body]).reset_axis(drop=True)
X_tvect_train.drop(columns=['body'], inplace=True)
X_tvect_test.drop(columns=['body'], inplace=True)

## ARRRRGGGHHH! - makes sense - cannot combine sparse matrix with dataframe or Series or array, etc.

TypeError: cannot concatenate object of type '<class 'scipy.sparse.csr.csr_matrix'>'; only Series and DataFrame objs are valid

In [9]:
X_tvect_body = tvect.fit_transform(X['body'])
X_new_tvect_body = tvect.transform(X_new['body'])

## Regression Models
#### Since the models are all converted to numeric form, trying out Linear Regression
This ends badly since I will have to round the predictions... but hey, why not try it!

<h2> (gridsearch) OneHotEncoder(), KNNImputer(), StandardScaler(), PolynomialFeatures(), ElasticNet() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [8]:
gs_enet = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
#     Ridge()), 
    ElasticNet()), 
    X_train, X_test, y_train, y_test,
    params={
#         'ridge__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__l1_ratio': [ 0.025, 0.05,  0.075, 0.1, 0.2, 0.3 ]
}, verbose=3, round_y_threshold=-1, invert_y=False, mod_name='Train/Test eNet')

gs_enet.check_y(y_test, gs_enet.y_pred)
print(gs_enet.model.best_params_)
# gs_enet.df

Fitting 5 folds for each of 24 candidates, totalling 120 fits


MemoryError: Unable to allocate 1.38 TiB for an array with shape (7370, 25743900) and data type float64

In [9]:
gs_enet_r = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
#     Ridge()), 
    ElasticNet()), 
    X_train, X_test, y_train, y_test,
    params={
#         'ridge__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__l1_ratio': [ 0.025, 0.05,  0.075, 0.1, 0.2, 0.3 ]
}, verbose=3, round_y_threshold=0.5, invert_y=False, mod_name='Train/Test eNet Rounding')

gs_enet_r.check_y(y_test, gs_enet_r.y_pred)
print(gs_enet_r.model.best_params_)
# gs_enet.df

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[[   0    1]
 [ 812 1645]]
{'elasticnet__alpha': 0.1, 'elasticnet__l1_ratio': 0.075}


In [10]:
X_gs_enet = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
#     Ridge()), 
    ElasticNet()), 
    X, X_new, y, y_new,
    params={
#         'ridge__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__l1_ratio': [ 0.025, 0.05,  0.075, 0.1, 0.2, 0.3 ]
}, verbose=3, round_y_threshold=-1, invert_y=False, mod_name='Xy/new eNet')

X_gs_enet.check_y(y_new, X_gs_enet.y_pred)
print(X_gs_enet.model.best_params_)
# X_gs_enet.df

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[[   0]
 [1962]]
{'elasticnet__alpha': 0.05, 'elasticnet__l1_ratio': 0.1}


In [11]:
X_gs_enet_r = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
#     Ridge()), 
    ElasticNet()), 
    X, X_new, y, y_new,
    params={
#         'ridge__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__alpha': [ 0.005, 0.01, 0.05, 0.1 ],
        'elasticnet__l1_ratio': [ 0.025, 0.05,  0.075, 0.1, 0.2, 0.3 ]
}, verbose=3, round_y_threshold=0.5, invert_y=False, mod_name='Xy/new eNet Rounding')

X_gs_enet_r.check_y(y_new, X_gs_enet_r.y_pred)
print(X_gs_enet_r.model.best_params_)
# X_gs_enet.df

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[[   0    1]
 [ 681 1281]]
{'elasticnet__alpha': 0.05, 'elasticnet__l1_ratio': 0.1}


In [12]:
compare_enet = pd.concat([gs_enet.df, gs_enet_r.df, X_gs_enet.df, X_gs_enet_r.df], axis=1)
compare_enet

,Train/Test eNet,Train/Test eNet Rounding,Xy/new eNet,Xy/new eNet Rounding
R2 Score,0.182128,-0.321939,0.167097,-0.388415
RMSE,0.452181,0.574878,0.456312,0.589148
MSE,0.204468,0.330484,0.208220,0.347095
MAE,0.417468,0.330484,0.423481,0.347095
Train R2 Score,0.166212,-0.367708,0.172344,-0.356672
Train RMSE,0.456560,0.584745,0.454878,0.582381
Train MSE,0.208447,0.341927,0.206914,0.339168
Train MAE,0.422775,0.341927,0.418257,0.339168


<h2> (gridsearch) OneHotEncoder(), KNNImputer(), StandardScaler(), PolynomialFeatures(), LinearRegression() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [13]:
gs_lr = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
    LinearRegression()), 
    X_train, X_test, y_train, y_test,
    params={
}, verbose=3, round_y_threshold=-1, invert_y=False, mod_name='Train/Test LinReg')

gs_lr.check_y(y_test, gs_lr.y_pred)
print(gs_lr.model.best_params_)
# gs_lr.df

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[[   0]
 [2457]]
{}


In [14]:
gs_lr_r = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
    LinearRegression()), 
    X_train, X_test, y_train, y_test,
    params={
}, verbose=3, round_y_threshold=0.5, invert_y=False, mod_name='Train/Test LinReg Rounding')

gs_lr_r.check_y(y_test, gs_lr_r.y_pred)
print(gs_lr_r.model.best_params_)
# gs_lr_r.df

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[[   0    1]
 [ 821 1636]]
{}


In [16]:
X_gs_lr = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
    LinearRegression()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
}, verbose=3, round_y_threshold=-1, invert_y=False, mod_name='Xy/new LinReg')

X_gs_lr.check_y(y_new, X_gs_lr.y_pred)
print(X_gs_lr.model.best_params_)
# X_gs_lr.df

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[[   0]
 [1962]]
{}


In [17]:
X_gs_lr_r = RegressionModel(make_pipeline(
    OneHotEncoder(),
    KNNImputer(), 
    StandardScaler(),
    PolynomialFeatures(),
    LinearRegression()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
}, verbose=3, round_y_threshold=-1, invert_y=False, mod_name='Xy/new LinReg Rounding')

X_gs_lr_r.check_y(y_new, X_gs_lr_r.y_pred)
print(X_gs_lr_r.model.best_params_)
# X_gs_lr_r.df

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[[   0]
 [1962]]
{}


In [18]:
compare_lr = pd.concat([gs_lr_r.df, gs_lr.df, gs_lr_r.df, gs_lr.df], axis=1)
compare_lr

,Train/Test LinReg Rounding,Train/Test LinReg,Train/Test LinReg Rounding,Train/Test LinReg
R2 Score,-0.336591,0.168731,-0.336591,0.168731
RMSE,0.578055,0.455869,0.578055,0.455869
MSE,0.334147,0.207817,0.334147,0.207817
MAE,0.334147,0.413928,0.334147,0.413928
Train R2 Score,-0.380191,0.165525,-0.380191,0.165525
Train RMSE,0.587407,0.456748,0.587407,0.456748
Train MSE,0.345047,0.208619,0.345047,0.208619
Train MAE,0.345047,0.418084,0.345047,0.418084


In [19]:
compare_regression = pd.concat([compare_enet, compare_lr], axis=1)
compare_regression

,Train/Test eNet,Train/Test eNet Rounding,Xy/new eNet,Xy/new eNet Rounding,Train/Test LinReg Rounding,Train/Test LinReg,Train/Test LinReg Rounding,Train/Test LinReg
R2 Score,0.182128,-0.321939,0.167097,-0.388415,-0.336591,0.168731,-0.336591,0.168731
RMSE,0.452181,0.574878,0.456312,0.589148,0.578055,0.455869,0.578055,0.455869
MSE,0.204468,0.330484,0.208220,0.347095,0.334147,0.207817,0.334147,0.207817
MAE,0.417468,0.330484,0.423481,0.347095,0.334147,0.413928,0.334147,0.413928
Train R2 Score,0.166212,-0.367708,0.172344,-0.356672,-0.380191,0.165525,-0.380191,0.165525
Train RMSE,0.456560,0.584745,0.454878,0.582381,0.587407,0.456748,0.587407,0.456748
Train MSE,0.208447,0.341927,0.206914,0.339168,0.345047,0.208619,0.345047,0.208619
Train MAE,0.422775,0.341927,0.418257,0.339168,0.345047,0.418084,0.345047,0.418084


## Classification Models


<h2> (gridsearch) RandomForestClassifier() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [20]:
gs_rfc = ClassificationModel(make_pipeline(
    RandomForestClassifier()), 
    X_train, X_test, y_train, y_test,
    params={
        'randomforestclassifier__criterion': [ 'gini', 'entropy' ],
        'randomforestclassifier__max_depth': [ None, 1, 2, 3 ],
        'randomforestclassifier__random_state': [ 3 ]
}, verbose=3, mod_name='Train/Test RFC')

print(gs_rfc.model.best_params_)
# gs_rfc.df

Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 3, 'randomforestclassifier__random_state': 3}


In [21]:
X_gs_rfc = ClassificationModel(make_pipeline(
    RandomForestClassifier()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'randomforestclassifier__criterion': [ 'gini', 'entropy' ],
        'randomforestclassifier__max_depth': [ None, 1, 2, 3 ],
        'randomforestclassifier__random_state': [ 3 ]
}, verbose=3, mod_name='Xy/new RFC')

print(X_gs_rfc.model.best_params_)
# X_gs_rfc.df

Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 3, 'randomforestclassifier__random_state': 3}


In [22]:
compare_rfc = pd.concat([gs_rfc.df, X_gs_rfc.df], axis=1)
compare_rfc

,Train/Test RFC,Xy/new RFC
F1 Score,0.681537,0.732052
Recall Score,0.670732,0.955943
Accuracy,0.686203,0.651886
Balanced Accuracy,0.686222,0.653428
Precision Score,0.692695,0.593134
Average Precision Score,0.748042,0.720120
ROC AUC Score,0.765069,0.751093
True Positive,861.000000,346.000000
False Negative,366.000000,640.000000
False Positive,405.000000,43.000000


<h2> (gridsearch) DecisionTreeClassifier() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [24]:
gs_tree = ClassificationModel(make_pipeline(
    DecisionTreeClassifier()), 
    X_train, X_test, y_train, y_test,
    params={
        'decisiontreeclassifier__max_depth': [ 1, 3, 5, None ],
        'decisiontreeclassifier__criterion': [ 'gini', 'entropy' ]
}, verbose=3, mod_name='Train/Test dTree')

# gs_tree.df

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [25]:
X_gs_tree = ClassificationModel(make_pipeline(
    DecisionTreeClassifier()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'decisiontreeclassifier__max_depth': [ 1, 3, 5, None ],
        'decisiontreeclassifier__criterion': [ 'gini', 'entropy' ]
}, verbose=3, mod_name='Xy/new dTree')

# X_gs_tree.df

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [26]:
compare_tree = pd.concat([gs_tree.df, X_gs_tree.df], axis=1)
compare_tree

,Train/Test dTree,Xy/new dTree
F1 Score,0.650000,0.595849
Recall Score,0.570732,0.500000
Accuracy,0.692308,0.662589
Balanced Accuracy,0.692456,0.661765
Precision Score,0.754839,0.737160
Average Precision Score,0.731528,0.706244
ROC AUC Score,0.773194,0.742770
True Positive,999.000000,812.000000
False Negative,228.000000,174.000000
False Positive,528.000000,488.000000


<h2> (gridsearch) BaggingClassifier() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [27]:
from sklearn.svm import SVC

In [28]:
gs_bag = ClassificationModel(make_pipeline(
    BaggingClassifier()), 
    X_train, X_test, y_train, y_test,
    params={
        'baggingclassifier__n_estimators': [ 5, 8, 10, 12 ,15 ],
        'baggingclassifier__base_estimator': [ None, SVC() ],
}, verbose=3, mod_name='Train/Test Bagging')

# gs_bag.df

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [29]:
X_gs_bag = ClassificationModel(make_pipeline(
    BaggingClassifier()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'baggingclassifier__n_estimators': [ 5, 8, 10, 12 ,15 ],
        'baggingclassifier__base_estimator': [ None, SVC() ],
}, verbose=3, mod_name='Xy/new Bagging')

# X_gs_bag.df

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [30]:
compare_bag = pd.concat([gs_bag.df, X_gs_bag.df], axis=1)
compare_bag

,Train/Test Bagging,Xy/new Bagging
F1 Score,0.661463,0.620690
Recall Score,0.654472,0.627049
Accuracy,0.664632,0.618756
Balanced Accuracy,0.664644,0.618798
Precision Score,0.668605,0.614458
Average Precision Score,0.687051,0.663335
ROC AUC Score,0.728134,0.688888
True Positive,828.000000,602.000000
False Negative,399.000000,384.000000
False Positive,425.000000,364.000000


<h2> (gridsearch) AdaBoostClassifier() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [32]:
gs_ada = ClassificationModel(make_pipeline(
    AdaBoostClassifier()), 
    X_train, X_test, y_train, y_test,
    params={
        'adaboostclassifier__n_estimators': [ 25, 50, 75, 100 ]
}, verbose=3, mod_name='Train/Test adaBst')

# gs_ada.df

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [33]:
X_gs_ada = ClassificationModel(make_pipeline(
    AdaBoostClassifier()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'adaboostclassifier__n_estimators': [ 25, 50, 75, 100 ]
}, verbose=3, mod_name='Xy/new adaBst')

# X_gs_ada.df

Fitting 5 folds for each of 4 candidates, totalling 20 fits


In [34]:
compare_ada = pd.concat([gs_ada.df, X_gs_ada.df], axis=1)
compare_ada

,Train/Test adaBst,Xy/new adaBst
F1 Score,0.665185,0.609256
Recall Score,0.608130,0.532787
Accuracy,0.693529,0.660041
Balanced Accuracy,0.693633,0.659395
Precision Score,0.734053,0.711354
Average Precision Score,0.745402,0.694213
ROC AUC Score,0.772003,0.733476
True Positive,956.000000,775.000000
False Negative,271.000000,211.000000
False Positive,482.000000,456.000000


<h2> (gridsearch) GradientBoostingClassifier() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [35]:
gs_gb = ClassificationModel(make_pipeline(
    GradientBoostingClassifier()), 
    X_train, X_test, y_train, y_test,
    params={
        'gradientboostingclassifier__n_estimators': [ 100 ],
        'gradientboostingclassifier__learning_rate': [ 0.1, 0.25, 0.5, 0.75, 1.0 ],
        'gradientboostingclassifier__max_depth': [ 1, 3, 5 ]
}, verbose=3, mod_name='Train/Test gBoost')

# gs_gb.df

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [36]:
X_gs_gb = ClassificationModel(make_pipeline(
    GradientBoostingClassifier()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'gradientboostingclassifier__n_estimators': [ 100 ],
        'gradientboostingclassifier__learning_rate': [ 0.1, 0.25, 0.5, 0.75, 1.0 ],
        'gradientboostingclassifier__max_depth': [ 1, 3, 5 ]
}, verbose=3, mod_name='Xy/new gBoost')

# X_gs_gb.df

Fitting 5 folds for each of 15 candidates, totalling 75 fits


In [37]:
compare_gb = pd.concat([gs_gb.df, X_gs_gb.df], axis=1)
compare_gb

,Train/Test gBoost,Xy/new gBoost
F1 Score,0.692461,0.652268
Recall Score,0.705691,0.618852
Accuracy,0.686203,0.671764
Balanced Accuracy,0.686179,0.671495
Precision Score,0.679718,0.689498
Average Precision Score,0.761952,0.733354
ROC AUC Score,0.775845,0.759623
True Positive,818.000000,714.000000
False Negative,409.000000,272.000000
False Positive,362.000000,372.000000


<h2> (gridsearch) LogisticRegression() </h2>

#### BEST Params:
 - max_features: 
 - ngram_range: 
 - stop_words: 

In [38]:
gs_lgr = ClassificationModel(make_pipeline(
    LogisticRegression()), 
    X_train, X_test, y_train, y_test,
    params={
        'logisticregression__C': [ 0.01, 0.1, 1, 10 ],
        'logisticregression__max_iter': [ 1000 ],
        'logisticregression__penalty': [ 'l1', 'l2', 'elasticnet', 'none' ]
}, verbose=3, mod_name='Train/Test LogReg')

# gs_lgr.df

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [39]:
X_gs_lgr = ClassificationModel(make_pipeline(
    LogisticRegression()), 
    X_train=X, X_test=X_new, y_train=y, y_test=y_new,
    params={
        'logisticregression__C': [ 0.01, 0.1, 1, 10 ],
        'logisticregression__max_iter': [ 1000 ],
        'logisticregression__penalty': [ 'l1', 'l2', 'elasticnet', 'none' ]
}, verbose=3, mod_name='Xy/new LogReg')

# X_gs_lgr.df

Fitting 5 folds for each of 16 candidates, totalling 80 fits


In [40]:
compare_lgr = pd.concat([gs_lgr.df, X_gs_lgr.df], axis=1)
compare_lgr

,Train/Test LogReg,Xy/new LogReg
F1 Score,0.740268,0.730572
Recall Score,0.943089,0.948770
Accuracy,0.668702,0.651886
Balanced Accuracy,0.668366,0.653391
Precision Score,0.609244,0.593970
Average Precision Score,0.708135,0.678817
ROC AUC Score,0.731256,0.686868
True Positive,483.000000,353.000000
False Negative,744.000000,633.000000
False Positive,70.000000,50.000000


<h2> Compare Test/Train Models </h2>

In [45]:
compare_train_df = pd.concat([gs_tree.df, gs_bag.df, gs_ada.df, gs_gb.df, gs_rfc.df, gs_lgr.df], axis=1)
compare_train_df

,Train/Test dTree,Train/Test Bagging,Train/Test adaBst,Train/Test gBoost,Train/Test RFC,Train/Test LogReg
F1 Score,0.650000,0.661463,0.665185,0.692461,0.681537,0.740268
Recall Score,0.570732,0.654472,0.608130,0.705691,0.670732,0.943089
Accuracy,0.692308,0.664632,0.693529,0.686203,0.686203,0.668702
Balanced Accuracy,0.692456,0.664644,0.693633,0.686179,0.686222,0.668366
Precision Score,0.754839,0.668605,0.734053,0.679718,0.692695,0.609244
Average Precision Score,0.731528,0.687051,0.745402,0.761952,0.748042,0.708135
ROC AUC Score,0.773194,0.728134,0.772003,0.775845,0.765069,0.731256
True Positive,999.000000,828.000000,956.000000,818.000000,861.000000,483.000000
False Negative,228.000000,399.000000,271.000000,409.000000,366.000000,744.000000
False Positive,528.000000,425.000000,482.000000,362.000000,405.000000,70.000000


<h2> Compare Xy/new Models </h2>

In [46]:
compare_X_df = pd.concat([X_gs_tree.df, X_gs_bag.df, X_gs_ada.df, X_gs_gb.df, X_gs_rfc.df, X_gs_lgr.df], axis=1)
compare_X_df

,Xy/new dTree,Xy/new Bagging,Xy/new adaBst,Xy/new gBoost,Xy/new RFC,Xy/new LogReg
F1 Score,0.595849,0.620690,0.609256,0.652268,0.732052,0.730572
Recall Score,0.500000,0.627049,0.532787,0.618852,0.955943,0.948770
Accuracy,0.662589,0.618756,0.660041,0.671764,0.651886,0.651886
Balanced Accuracy,0.661765,0.618798,0.659395,0.671495,0.653428,0.653391
Precision Score,0.737160,0.614458,0.711354,0.689498,0.593134,0.593970
Average Precision Score,0.706244,0.663335,0.694213,0.733354,0.720120,0.678817
ROC AUC Score,0.742770,0.688888,0.733476,0.759623,0.751093,0.686868
True Positive,812.000000,602.000000,775.000000,714.000000,346.000000,353.000000
False Negative,174.000000,384.000000,211.000000,272.000000,640.000000,633.000000
False Positive,488.000000,364.000000,456.000000,372.000000,43.000000,50.000000
